### Import libraries

In [25]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import arviz as az

import IPython

from meridian import constants
from meridian.data import load
from meridian.data import test_utils
from meridian.model import model
from meridian.model import spec
from meridian.model import prior_distribution
from meridian.analysis import optimizer
from meridian.analysis import analyzer
from meridian.analysis import visualizer
from meridian.analysis import summarizer
from meridian.analysis import formatter

### Define variables

In [2]:
# file paths
DATA_FOLDER = "data/ME_FINISH q4 2024"
input_params_path = f'{DATA_FOLDER}/input_files/Input params.xlsx'
collected_data_path = f'{DATA_FOLDER}/meridian/input_files/Meridian data collection template.xlsx'
input_df_path = f'{DATA_FOLDER}/meridian/intermediate_files/input_df.csv'
campaign_df_path = f'{DATA_FOLDER}/meridian/intermediate_files/campaign_level/input_df_camp_' + '{0}.csv'

In [3]:
DATE = 'date'
START_IMPR_PREFIX = 'Impressions '
START_SPEND_PREFIX = 'Spend '
SPEND_PREFIX = 'mdsp_'
IMPR_PREFIX = 'mdip_'
CTRB_PREFIX = 'mdct_'
TARGET = 'sales_units'
REVENUE_PER_KPI = 'price'

controls = ['tdp_sum', 'promo_avg', 'price', 'competition', 'seas_yearly']

In [4]:
from data_processing.config_generation import get_params_from_excel

params = get_params_from_excel(input_params_path)
priors_channel = params['ROI_INTERVALS']
priors_campaign = params['ROIS_SEGMENT']

In [5]:
roi_start_date = '2023-01-01'
roi_end_date = '2023-12-31'
knots_frequency = 0.8

In [6]:
sample_prior = 500
n_chains=7
n_adapt=500
n_burnin=500
n_keep=1000

### Read and process data

In [7]:
def read_data_from_excel(sheet_name):
  df = pd.read_excel(collected_data_path, sheet_name=sheet_name)
  df = df.drop(df.columns[0], axis=1).rename(columns={'Unnamed: 1': DATE})
  df[DATE] = pd.to_datetime(df[DATE])
  df.set_index(DATE, inplace=True)
  df.rename(columns={col: col.replace(START_IMPR_PREFIX, IMPR_PREFIX).replace(START_SPEND_PREFIX, SPEND_PREFIX) for col in df.columns}, inplace=True)
  return df

In [8]:
df_ctrl = read_data_from_excel("B&M non media drivers")
df_chan = read_data_from_excel("B&M channel level")

In [9]:
df = pd.concat([df_ctrl, df_chan], axis=1)
df.to_csv(input_df_path)

### Define model parameters

Load the data

In [10]:
impr_columns = [col for col in df.columns if col.startswith(IMPR_PREFIX)]
spend_columns = [col for col in df.columns if col.startswith(SPEND_PREFIX)]
media_columns = [col.replace(IMPR_PREFIX, '') for col in impr_columns]

coord_to_columns = load.CoordToColumns(
    time=DATE,
    # geo='geo',
    controls=controls,
    # population='population',
    kpi=TARGET,
    revenue_per_kpi=REVENUE_PER_KPI,
    media=impr_columns,
    media_spend=spend_columns,
)

correct_media_to_channel = {col: col.replace(IMPR_PREFIX, '') for col in impr_columns}
correct_media_spend_to_channel = {col: col.replace(SPEND_PREFIX, '') for col in spend_columns}

In [11]:
loader = load.CsvDataLoader(
    csv_path=input_df_path,
    kpi_type='non_revenue',
    coord_to_columns=coord_to_columns,
    media_to_channel=correct_media_to_channel,
    media_spend_to_channel=correct_media_spend_to_channel,
)
data = loader.load()

/Users/datascienceua/Documents/meridian/meridian/data/load.py:1020: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df[geo_column_name] = self.df[geo_column_name].replace(
/Users/datascienceua/Documents/meridian/meridian/data/load.py:201: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  if (constants.GEO) not in self.dataset.dims.keys():
/Users/datascienceua/Documents/meridian/meridian/data/load.py:206: UserWarning: The `population` argument is ignored in a nationally aggregated model. It will be reset to [1]
  warnings.warn(
/Users/datascienceua/Documents/mer

Set the priors

In [12]:
build_media_channel_args = data.get_paid_media_channels_argument_builder()

roi_m = build_media_channel_args(**priors_channel) # This creates a list of channel-ordered (mu, sigma) tuples.
roi_m_low, roi_m_high = zip(*roi_m)

prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.Uniform(
        roi_m_low, roi_m_high, name=constants.ROI_M
    )
)

In [13]:
# roi_mu = 0.2     # Mu for ROI prior for each media channel.
# roi_sigma = 0.9  # Sigma for ROI prior for each media channel.
# prior = prior_distribution.PriorDistribution(
#     roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name=constants.ROI_M)
# )

Set the ROI calibration period

In [14]:
roi_dates = [date for date in data.time.values if roi_start_date <= date <= roi_end_date]
roi_period = {
  channel: roi_dates if data.media.loc[:,roi_dates,channel].sum()!=0 else data.time.values.tolist() for channel in data.media_channel.values
}

roi_calibration_period = np.zeros((len(data.time), len(data.media_channel)))
for i in roi_period.items():
  roi_calibration_period[
      np.isin(data.time.values, i[1]), data.media_channel.values == i[0]
  ] = 1

roi_calibration_period[
    :, ~np.isin(data.media_channel.values, list(roi_period.keys()))
] = 1

Tune the automatic seasonality adjustment

In [15]:
knots = round(knots_frequency * len(data.time.values))

Configure the model

In [16]:
model_spec = spec.ModelSpec(prior=prior, knots=knots, roi_calibration_period=roi_calibration_period)

mmm = model.Meridian(input_data=data, model_spec=model_spec)

/Users/datascienceua/Documents/meridian/meridian/model/model.py:64: UserWarning: In a nationally aggregated model, the `media_effects_dist` will be reset to `normal`.
  warnings.warn(
I0000 00:00:1742463483.716894 19064310 service.cc:145] XLA service 0x2935e0670 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1742463483.716914 19064310 service.cc:153]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1742463483.725919 19064310 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [17]:
mmm.sample_prior(sample_prior)
mmm.sample_posterior(n_chains=n_chains, n_adapt=n_adapt, n_burnin=n_burnin, n_keep=n_keep)

/Users/datascienceua/Documents/meridian/meridian/model/prior_distribution.py:846: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. tau_g_excl_baseline has been automatically set to Deterministic(0).
  warnings.warn(
/Users/datascienceua/Documents/meridian/meridian/model/prior_distribution.py:846: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. eta_m has been automatically set to Deterministic(0).
  warnings.warn(
/Users/datascienceua/Documents/meridian/meridian/model/prior_distribution.py:846: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. eta_rf has been automatically set to Deterministic(0).
  warnings.warn(
/Users/datascienceua/Documents/meridian/meridian/model/prior_distribution.py:846: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. eta_om has been automatically set to

Diagnose the model

In [18]:
model_diagnostics = visualizer.ModelDiagnostics(mmm)
model_diagnostics.plot_rhat_boxplot()

/Users/datascienceua/Documents/meridian/.venv/lib/python3.12/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

In [19]:
model_fit = visualizer.ModelFit(mmm)
model_fit.plot_model_fit()

/Users/datascienceua/Documents/meridian/meridian/analysis/analyzer.py:498: UserWarning: The `aggregate_geos` argument is ignored in the national model. It will be reset to `True`.
  warnings.warn(
/Users/datascienceua/Documents/meridian/.venv/lib/python3.12/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/datascienceua/Documents/meridian/.venv/lib/python3.12/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

Extract the results and create campaign datasets

In [21]:
df_camp = {channel: read_data_from_excel(f"B&M campaign level - {channel}") for channel in media_columns}

In [49]:
model_analyzer = analyzer.Analyzer(mmm)

outcome_tensor = model_analyzer.incremental_outcome(aggregate_times=False, use_kpi=True, include_non_paid_channels=True, non_media_baseline_values=True)
out_df = pd.DataFrame(tf.reduce_mean(outcome_tensor, axis=[0,1]).numpy())
# out_df.columns = media_columns

In [51]:
outcome_tensor

<tf.Tensor: shape=(7, 1000, 104, 4), dtype=float32, numpy=
array([[[[   0.      ,    0.      ,    0.      ,    0.      ],
         [   0.      ,    0.      ,    0.      ,    0.      ],
         [   0.      ,    0.      ,    0.      ,    0.      ],
         ...,
         [   0.      , 1067.3281  ,  719.96094 ,    0.      ],
         [   0.      , 1128.2031  ,  820.6836  ,    0.      ],
         [   0.      , 1121.9043  ,  875.7969  ,    0.      ]],

        [[   0.      ,    0.      ,    0.      ,    0.      ],
         [   0.      ,    0.      ,    0.      ,    0.      ],
         [   0.      ,    0.      ,    0.      ,    0.      ],
         ...,
         [   0.      , 1125.3516  , 1687.7148  ,    0.      ],
         [   0.      , 1365.9414  , 1875.959   ,    0.      ],
         [   0.      , 1330.3594  , 1917.752   ,    0.      ]],

        [[   0.      ,    0.      ,    0.      ,    0.      ],
         [   0.      ,    0.      ,    0.      ,    0.      ],
         [   0.      ,    0

In [50]:
out_df

,0,1,2,3
0,0.0,0.000000,0.000000,0.0
1,0.0,0.000000,0.000000,0.0
2,0.0,0.000000,0.000000,0.0
3,0.0,287.329926,0.000000,0.0
4,0.0,422.448975,0.000000,0.0
...,...,...,...,...
99,0.0,1047.027100,879.142029,0.0
100,0.0,888.150696,543.331909,0.0
101,0.0,1092.347534,1219.157471,0.0
102,0.0,1241.926147,1350.112915,0.0


In [52]:
model_analyzer.baseline_summary_metrics(aggregate_times=False).to_dataframe()

baseline_outcome   channel  \
time       metric distribution                               
2023-01-07 mean   prior             8.934769e+05  baseline   
                  posterior         8.231825e+05  baseline   
           median prior             7.218825e+05  baseline   
                  posterior         8.227149e+05  baseline   
           ci_lo  prior            -6.611665e+06  baseline   
...                                          ...       ...   
2024-12-28 median posterior         7.564339e+05  baseline   
           ci_lo  prior            -3.193154e+06  baseline   
                  posterior         6.733082e+05  baseline   
           ci_hi  prior             4.833480e+06  baseline   
                  posterior         8.419576e+05  baseline   

                                pct_of_contribution  
time       metric distribution                       
2023-01-07 mean   prior                   99.999954  
                  posterior              100.000084  
           median prior                   80.794746  
                  posterior               99.943619  
           ci_lo  prior                 -739.992780  
...                                             ...  
2024-12-28 median posterior               87.601128  
           ci_lo  prior                 -348.963113  
                  posterior               77.974507  
           ci_hi  prior                  528.225854  
                  posterior               97.505463  

[832 rows x 3 columns]

In [42]:
out_df

,SOCIAL,TV,TV_New,VIDEO
0,0.0,0.000000,0.000000,0.0
1,0.0,0.000000,0.000000,0.0
2,0.0,0.000000,0.000000,0.0
3,0.0,287.329926,0.000000,0.0
4,0.0,422.448975,0.000000,0.0
...,...,...,...,...
99,0.0,1047.027100,879.142029,0.0
100,0.0,888.150696,543.331909,0.0
101,0.0,1092.347534,1219.157471,0.0
102,0.0,1241.926147,1350.112915,0.0


In [34]:
model_analyzer.predictive_accuracy().to_dataframe()

/Users/datascienceua/Documents/meridian/meridian/analysis/analyzer.py:498: UserWarning: The `aggregate_geos` argument is ignored in the national model. It will be reset to `True`.
  warnings.warn(


,,value
metric,geo_granularity,
R_Squared,national,0.996317
MAPE,national,0.014156
wMAPE,national,0.012249


In [36]:
out_df.sum()

SOCIAL     1442.937134
TV        61408.824219
TV_New    11685.181641
VIDEO     11139.002930
dtype: float32

In [56]:
non_media_baseline_values = ["min" for col in controls]

In [60]:
model_analyzer.baseline_summary_metrics().to_dataframe()

baseline_outcome   channel  pct_of_contribution
metric distribution                                                 
mean   prior               80081272.0  baseline            95.372437
       posterior           75284472.0  baseline            95.235855
median prior               81057008.0  baseline            96.534477
       posterior           75285136.0  baseline            95.236694
ci_lo  prior               24327189.5  baseline            28.972359
       posterior           74115555.6  baseline            93.757166
ci_hi  prior              138405629.6  baseline           164.833564
       posterior           76442172.4  baseline            96.700361

In [58]:
model_analyzer.summary_metrics(use_kpi=True).to_dataframe()

impressions  pct_of_impressions  \
channel      metric distribution                                     
SOCIAL       mean   prior         2.947903e+07           17.117245   
                    posterior     2.947903e+07           17.117245   
             median prior         2.947903e+07           17.117245   
                    posterior     2.947903e+07           17.117245   
             ci_lo  prior         2.947903e+07           17.117245   
                    posterior     2.947903e+07           17.117245   
             ci_hi  prior         2.947903e+07           17.117245   
                    posterior     2.947903e+07           17.117245   
TV           mean   prior         2.770067e+04            0.016085   
                    posterior     2.770067e+04            0.016085   
             median prior         2.770067e+04            0.016085   
                    posterior     2.770067e+04            0.016085   
             ci_lo  prior         2.770067e+04            0.016085   
                    posterior     2.770067e+04            0.016085   
             ci_hi  prior         2.770067e+04            0.016085   
                    posterior     2.770067e+04            0.016085   
TV_New       mean   prior         6.296866e+02            0.000366   
                    posterior     6.296866e+02            0.000366   
             median prior         6.296866e+02            0.000366   
                    posterior     6.296866e+02            0.000366   
             ci_lo  prior         6.296866e+02            0.000366   
                    posterior     6.296866e+02            0.000366   
             ci_hi  prior         6.296866e+02            0.000366   
                    posterior     6.296866e+02            0.000366   
VIDEO        mean   prior         1.427110e+08           82.866302   
                    posterior     1.427110e+08           82.866302   
             median prior         1.427110e+08           82.866302   
                    posterior     1.427110e+08           82.866302   
             ci_lo  prior         1.427110e+08           82.866302   
                    posterior     1.427110e+08           82.866302   
             ci_hi  prior         1.427110e+08           82.866302   
                    posterior     1.427110e+08           82.866302   
All Channels mean   prior         1.722183e+08          100.000000   
                    posterior     1.722183e+08          100.000000   
             median prior         1.722183e+08          100.000000   
                    posterior     1.722183e+08          100.000000   
             ci_lo  prior         1.722183e+08          100.000000   
                    posterior     1.722183e+08          100.000000   
             ci_hi  prior         1.722183e+08          100.000000   
                    posterior     1.722183e+08          100.000000   

                                         spend  pct_of_spend           cpm  \
channel      metric distribution                                             
SOCIAL       mean   prior         4.555684e+04      1.081257  1.545398e+00   
                    posterior     4.555684e+04      1.081257  1.545398e+00   
             median prior         4.555684e+04      1.081257  1.545398e+00   
                    posterior     4.555684e+04      1.081257  1.545398e+00   
             ci_lo  prior         4.555684e+04      1.081257  1.545398e+00   
                    posterior     4.555684e+04      1.081257  1.545398e+00   
             ci_hi  prior         4.555684e+04      1.081257  1.545398e+00   
                    posterior     4.555684e+04      1.081257  1.545398e+00   
TV           mean   prior         2.677130e+06     63.539608  9.664495e+04   
                    posterior     2.677130e+06     63.539608  9.664495e+04   
             median prior         2.677130e+06     63.539608  9.664495e+04   
                    posterior     2.677130e+06     63.539608  9.664495e+04   
  

In [53]:
out_df.sum()

SOCIAL     1442.937134
TV        61408.824219
TV_New    11685.181641
VIDEO     11139.002930
dtype: float32

In [26]:
for channel in media_columns:
  df_camp[channel][f'{CTRB_PREFIX}{channel}'] = out_df[channel].values
  df_camp[channel][REVENUE_PER_KPI] = df[REVENUE_PER_KPI].values
  df_camp[channel].to_csv(campaign_df_path.format(channel))

Campaign level model

In [29]:
class CampaignModel():
  def __init__(self, channel):
    self.channel = channel
    df = pd.read_csv(campaign_df_path.format(channel))

    impr_columns = [col for col in df.columns if col.startswith(IMPR_PREFIX)]
    spend_columns = [col for col in df.columns if col.startswith(SPEND_PREFIX)]
    media_columns = [col.replace(IMPR_PREFIX, '') for col in impr_columns]

    coord_to_columns = load.CoordToColumns(
        time=DATE,
        # geo='geo',
        controls=[],
        # population='population',
        kpi=f'{CTRB_PREFIX}{channel}',
        revenue_per_kpi=REVENUE_PER_KPI,
        media=impr_columns,
        media_spend=spend_columns,
    )

    correct_media_to_channel = {col: col.replace(IMPR_PREFIX, '') for col in impr_columns}
    correct_media_spend_to_channel = {col: col.replace(SPEND_PREFIX, '') for col in spend_columns}
    self.loader = load.CsvDataLoader(
        csv_path=campaign_df_path.format(channel),
        kpi_type='non_revenue',
        coord_to_columns=coord_to_columns,
        media_to_channel=correct_media_to_channel,
        media_spend_to_channel=correct_media_spend_to_channel,
    )
    self.data = self.loader.load()

    # roi_mu = 0.2     # Mu for ROI prior for each media channel.
    # roi_sigma = 0.9  # Sigma for ROI prior for each media channel.
    # self.prior = prior_distribution.PriorDistribution(
    #     roi_m=tfp.distributions.LogNormal(
    #         roi_mu,
    #         roi_sigma,
    #         name=constants.ROI_M,
    #     )
    # )
    build_media_channel_args = self.data.get_paid_media_channels_argument_builder()

    roi_m = build_media_channel_args(**priors_campaign[channel]) # This creates a list of channel-ordered (mu, sigma) tuples.
    roi_m_low, roi_m_high = zip(*roi_m)

    self.prior = prior_distribution.PriorDistribution(
        roi_m=tfp.distributions.Uniform(
            roi_m_low, roi_m_high, name=constants.ROI_M
        )
    )
    self.model_spec = spec.ModelSpec(prior=self.prior)
    self.mmm = model.Meridian(input_data=self.data, model_spec=self.model_spec)

    self.mmm.sample_prior(sample_prior)
    self.mmm.sample_posterior(n_chains=n_chains, n_adapt=n_adapt, n_burnin=n_burnin, n_keep=n_keep)

In [30]:
models = {}
for channel in media_columns:
  models[channel] = CampaignModel(channel)

/Users/datascienceua/Documents/meridian/meridian/data/load.py:1020: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df[geo_column_name] = self.df[geo_column_name].replace(
/Users/datascienceua/Documents/meridian/meridian/data/load.py:201: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  if (constants.GEO) not in self.dataset.dims.keys():
/Users/datascienceua/Documents/meridian/meridian/data/load.py:206: UserWarning: The `population` argument is ignored in a nationally aggregated model. It will be reset to [1]
  warnings.warn(
/Users/datascienceua/Documents/mer

/Users/datascienceua/Documents/meridian/meridian/data/load.py:1020: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df[geo_column_name] = self.df[geo_column_name].replace(
/Users/datascienceua/Documents/meridian/meridian/data/load.py:201: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  if (constants.GEO) not in self.dataset.dims.keys():
/Users/datascienceua/Documents/meridian/meridian/data/load.py:206: UserWarning: The `population` argument is ignored in a nationally aggregated model. It will be reset to [1]
  warnings.warn(
/Users/datascienceua/Documents/mer

W0000 00:00:1742396237.789461 18771800 assert_op.cc:38] Ignoring Assert operator mcmc_retry_init/assert_equal_1/Assert/AssertGuard/Assert


In [ ]:
for channel in media_columns:
  model_fit = visualizer.ModelFit(models[channel].mmm)
  model_fit.plot_model_fit()

In [ ]:
model_fit = visualizer.ModelFit(models["TV_New"].mmm)
model_fit.plot_model_fit()

In [34]:
model_analyzer = analyzer.Analyzer(models["TV_New"].mmm)

In [ ]:
model_analyzer.predictive_accuracy()

In [ ]:
model_analyzer.expected_vs_actual_data()

In [ ]:
media_summary = visualizer.MediaSummary(models["TV_New"].mmm)
media_summary.summary_table()

In [36]:
mmm_summarizer = summarizer.Summarizer(models["TV_New"].mmm)

In [ ]:
filepath = f'{DATA_FOLDER}/meridian'
filename = 'summary_output.html'
start_date = df.index.min()
end_date = df.index.max()
mmm_summarizer.output_model_results_summary(filename, filepath, start_date, end_date)